# Projeto 1 - Ciência dos Dados

Nome: Victor de Almeida Cunha

Nome: André Samara Levorin

Nome: Caio Cavalcante Bueno

___
Carregando algumas bibliotecas:

In [399]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [400]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Victor Cunha\Desktop\Atividades\C-Dados\CDADOS - DP\projeto1-cdados


Carregando a base de dados com os tweets classificados manualmente:

In [401]:
train = pd.read_excel('dados_treino_CLASSIFICADO.xlsx')
train.head(5)

,Mensagem,Target
0,"Não cheguei a completar a leitura do livro, te...",2
1,"Péssimo. Qualidade horrível do papel, tamanho ...",2
2,Fiz a compra do produto e chegou tudo certo co...,0
3,"Vetusto, tanto quanto o próprio autor, comprar...",2
4,Se soubesse não teria comprado. É bom pra apre...,2


In [402]:
test = pd.read_excel('dados_teste_CLASSIFICADO.xlsx')
test.head(5)

,Mensagem,Target
0,"Livro ruim, comparado com seu antecessor! A au...",2
1,O Livro é um resumo do canal Me Poupe! O Conte...,2
2,Desnecessário comentar. Não percam tempo. Níve...,2
3,Essa encomenda não veio pelos correios e chego...,0
4,"Assim como todos os livro da Série, não curti....",2


___
## Classificador automático


O intuito do classificador que montaremos é de usar conceitos e análises probabilísticas para fazer com que uma avaliação automática chegue o mais perto possível da avaliação manual de cada mensagem do Dataframe, utilizando as palavras como referência principal.

___
### Separando as probabilidades do Dataframe de treino

Para início, vamos importar algumas funções de limpeza para facilitar o entendimento de cada avaliação.

In [403]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[´"!-.:?;$'']' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def limpaStopwords(phrase):

    # Importação de uma lista de stopwords para minimizar palavras desnecessárias.

    stop_words = ['e', 'a', 'o','de','da', 'em','do', 'as', 'dos', 'das', 'no', 'na', 'quem', 'ao', 'que', 'sua','seu', 'por', 'uma', 
    'um', 'os', 'se','sobre', 'mas','são','entre', 'assim', 'esse', 'essa', 'diz', 'deve', 'anos', 'dia', 't', 'mi', 'nesta','neste','nesse','nessa', 'aos', 'com', 'já', 'para', 'mais','após','no','pelo', 'antes','depois', 'nos',
    'é', 'pela', 'desta','se','sobre', 'mas','são', 'que']

    # Dividiremos a review em cada palavra e removeremos todas as que são stopwords.
    
    frase = phrase.split()
    sem_stop = ''
    for word in frase:
        if word in stop_words:
            frase.remove(word)
        else:
            sem_stop += word + ' '
    return sem_stop

O próximo passo é separar o nosso Dataframe de treino em 3 diferentes (um para cada tipo de avaliação), além de um apenas com as mensagens.

In [417]:
# Dataframe com as avaliações positivas

train_pos = train.loc[train['Target'] == 1, ['Mensagem']]

# Dataframe com as avaliações negativas

train_neg = train.loc[train['Target'] == 2, ['Mensagem']]

# Dataframe com as avaliações irrelevantes

train_irr = train.loc[train['Target'] == 0, ['Mensagem']]

# Dataframe apenas com as mensagens

train_all = train.loc[:,['Mensagem']]

,Mensagem
0,"Não cheguei a completar a leitura do livro, te..."
1,"Péssimo. Qualidade horrível do papel, tamanho ..."
2,Fiz a compra do produto e chegou tudo certo co...
3,"Vetusto, tanto quanto o próprio autor, comprar..."
4,Se soubesse não teria comprado. É bom pra apre...


Agora, iremos fazer diferentes cálculos de probabilidade. De começo, iremos ver a chance de uma palavra aparecer em uma avaliação qualquer.

In [418]:
# Primeiro, criamos uma lista com todas as palavras do Dataframe original.

all_words = []

for review in train_all['Mensagem']:

    # Limparemos cada uma das mensagens para que a identificação fique correta.

    review = cleanup(review)
    review = limpaStopwords(review)

    # Agora, dividiremos em uma lista apenas com as palavras.

    words_review = review.split()
    for word in words_review:

        # Adicionaremos à lista apenas palavras únicas, sem serem repetidas.

        if word not in all_words:
            all_words.append(word)

# O mesmo processo será feito agora pra um dicionário, que terá a palavra como chave e a probabilidade dela aparecer
# em uma mensagem como valor.

word_probability_total = {}

for word in all_words:
    count_word = 0
    for review in train_all['Mensagem']:
        review = cleanup(review)
        review = limpaStopwords(review)
        words_review = review.split()
        if word in words_review:
            count_word += 1

    # Aqui, estamos dividindo em quantas mensagens a palavra apareceu pelo total de mensagens, e adicionando o valor ao dicionário.

    prob_appearance = count_word/len(train_all['Mensagem'])
    word_probability_total[word] = prob_appearance

Em seguida, repetiremos o mesmo processo, mas para encontrar qual é a chance de uma palavra em uma avaliação positiva aparecer em uma avaliação positiva qualquer.

In [420]:
all_words_positive = []

for review in train_pos['Mensagem']:
    review = cleanup(review)
    review = limpaStopwords(review)
    words_review = review.split()
    for word in words_review:
        if word not in all_words_positive:
            all_words_positive.append(word)

word_probability_positive = {}

for word in all_words_positive:
    count_word = 0
    for review in train_pos['Mensagem']:
        review = cleanup(review)
        review = limpaStopwords(review)
        words_review = review.split()
        if word in words_review:
            count_word += 1

    # A diferença principal das repetições seguintes é que o total agora é de mensagens com o tipo de avaliação.
    
    prob_appearance = count_word/len(train_pos['Mensagem'])
    word_probability_positive[word] = prob_appearance

E então, o processo é repetido para as avaliações negativas e irrelevantes.

In [421]:
all_words_negative = []

for review in train_neg['Mensagem']:
    review = cleanup(review)
    review = limpaStopwords(review)
    words_review = review.split()
    for word in words_review:
        if word not in all_words_negative:
            all_words_negative.append(word)

word_probability_negative = {}

for word in all_words_negative:
    count_word = 0
    for review in train_neg['Mensagem']:
        review = cleanup(review)
        review = limpaStopwords(review)
        words_review = review.split()
        if word in words_review:
            count_word += 1
    prob_appearance = count_word/len(train_neg['Mensagem'])
    word_probability_negative[word] = prob_appearance

In [422]:
all_words_irrelevant = []

for review in train_irr['Mensagem']:
    review = cleanup(review)
    review = limpaStopwords(review)
    words_review = review.split()
    for word in words_review:
        if word not in all_words_irrelevant:
            all_words_irrelevant.append(word)

word_probability_irrelevant = {}

for word in all_words_irrelevant:
    count_word = 0
    for review in train_irr['Mensagem']:
        review = cleanup(review)
        review = limpaStopwords(review)
        words_review = review.split()
        if word in words_review:
            count_word += 1
    prob_appearance = count_word/len(train_irr['Mensagem'])
    word_probability_irrelevant[word] = prob_appearance

Agora, veremos a possibilidade de uma avaliação qualquer ser positiva, negativa ou irrelevante.

In [423]:
# O cálculo de probabilidade de cada tipo de avaliação
# é feito por dividir quantas mensagens de cada tipo de avaliação existem
# pelo total de mensagens.

positive_review_probability = len(train_pos['Mensagem'])/len(train_all['Mensagem'])
negative_review_probability = len(train_neg['Mensagem'])/len(train_all['Mensagem'])
irrelevant_review_probability = len(train_irr['Mensagem'])/len(train_all['Mensagem'])

0.138
0.652
0.21


Com todas essas informações, passaremos para o processo de montar o classificador.

___
### Montando o Classificador de treino

Para começar a montagem do classificador, teremos que primeiramente usar as funções de limpeza para facilitar a identificação. Após, iremos montar probabilidades condicionais com base em todas as variáveis que montamos que mostram as probabilidades de cada tipo de avaliação.

A função básica de condicional que iremos usar é:


$$P(Review|word_n) = P(word_n|Review)/P(word_n)$$


Pode perceber que nessa função, está faltando a multiplicação pelo condicional de ser uma avaliação daquele tipo. Chegaremos nesse ponto ainda.

Iterando e multiplicando uma operação dessas sobre a seguinte, de forma igual, teremos o seguinte formato de fórmula:

$$P(Review|word_1,word_2,...,word_n) = P(word_1,word_2,...,word_n|Review)/P(word_1,word_2,...,word_n)$$

Com isso, basta multiplicar o resultado final pelo tipo de avaliação que se deseja observar, ou seja, multiplicar por $P(Review)$.

Após isso, comparamos qual é a maior probabilidade, e aquele que for maior, terá o valor atribuído à mensagem.

In [426]:
def Classificador_Treino(phrase) -> int:

    # Começamos por limpar as mensagens.

    phrase = cleanup(phrase)
    phrase = limpaStopwords(phrase)
    words_in_review = phrase.split()
    prob_positive = 1
    prob_negative = 1
    prob_irrelevant = 1

    # Então, fazemos o cálculo probabilístico mostrado acima.

    for word in words_in_review:
        if word in all_words_positive:
            prob_positive *= word_probability_positive[word]/word_probability_total[word]
        if word in all_words_negative:
            prob_negative *= word_probability_negative[word]/word_probability_total[word]
        if word in all_words_irrelevant:
            prob_irrelevant *= word_probability_irrelevant[word]/word_probability_total[word]

    prob_positive *= positive_review_probability
    prob_negative *= negative_review_probability
    prob_irrelevant *= irrelevant_review_probability

    # Aqui, aplicamos o conceito dito no final: se o valor máximo da
    # lista for uma avaliação positiva, retorna 1; negativa, 2; irrelevante, 0.

    chances = [prob_positive,prob_negative,prob_irrelevant]
    if prob_positive == max(chances):
        return 1
    if prob_negative == max(chances):
        return 2
    if prob_irrelevant == max(chances):
        return 0

Com a função montada, basta então aplicar no Dataframe inteiro de treino.

In [427]:
bot_train = []
for review in train_all['Mensagem']:

    # Nessa iteração, estamos aplicando o classificador em cada mensagem do Dataframe original.

    classification = Classificador_Treino(review)
    bot_train.append(classification)
pd.crosstab(train['Target'],bot_train,normalize=True)*100

col_0,0,1,2
Target,,,
0,20.6,0.4,0.0
1,0.0,13.8,0.0
2,2.8,14.8,47.6


Ao comparar as classificações que o classificador montou com as verdadeiras, que são os valores na diagonal, é possível observar que a precisão no treino foi de aproximadamente 82%.

___
### Verificando a performance do Classificador

Hora de colocar o classificador pelo teste de fogo.

Para isso, iremos repetir o mesmo processo que fizemos anteriormente, agora para o Dataframe de testes.

In [428]:
test_pos = test.loc[test['Target'] == 1, ['Mensagem']]
test_neg = test.loc[test['Target'] == 2, ['Mensagem']]
test_irr = test.loc[test['Target'] == 0, ['Mensagem']]
test_all = test.loc[:,['Mensagem']]

In [429]:
all_words = []

for review in test_all['Mensagem']:
    review = cleanup(review)
    review = limpaStopwords(review)
    words_review = review.split()
    for word in words_review:
        if word not in all_words:
            all_words.append(word)

word_probability_total = {}

for word in all_words:
    count_word = 0
    for review in test_all['Mensagem']:
        review = cleanup(review)
        review = limpaStopwords(review)
        words_review = review.split()
        if word in words_review:
            count_word += 1
    prob_appearance = count_word/len(test_all['Mensagem'])
    word_probability_total[word] = prob_appearance

In [430]:
all_words_positive = []

for review in test_pos['Mensagem']:
    review = cleanup(review)
    review = limpaStopwords(review)
    words_review = review.split()
    for word in words_review:
        if word not in all_words_positive:
            all_words_positive.append(word)

word_probability_positive = {}

for word in all_words_positive:
    count_word = 0
    for review in test_pos['Mensagem']:
        review = cleanup(review)
        review = limpaStopwords(review)
        words_review = review.split()
        if word in words_review:
            count_word += 1
    prob_appearance = count_word/len(test_pos['Mensagem'])
    word_probability_positive[word] = prob_appearance

In [431]:
all_words_negative = []

for review in test_neg['Mensagem']:
    review = cleanup(review)
    review = limpaStopwords(review)
    words_review = review.split()
    for word in words_review:
        if word not in all_words_negative:
            all_words_negative.append(word)

word_probability_negative = {}

for word in all_words_negative:
    count_word = 0
    for review in test_neg['Mensagem']:
        review = cleanup(review)
        review = limpaStopwords(review)
        words_review = review.split()
        if word in words_review:
            count_word += 1
    prob_appearance = count_word/len(test_neg['Mensagem'])
    word_probability_negative[word] = prob_appearance

In [432]:
all_words_irrelevant = []

for review in test_irr['Mensagem']:
    review = cleanup(review)
    review = limpaStopwords(review)
    words_review = review.split()
    for word in words_review:
        if word not in all_words_irrelevant:
            all_words_irrelevant.append(word)

word_probability_irrelevant = {}

for word in all_words_irrelevant:
    count_word = 0
    for review in test_irr['Mensagem']:
        review = cleanup(review)
        review = limpaStopwords(review)
        words_review = review.split()
        if word in words_review:
            count_word += 1
    prob_appearance = count_word/len(test_irr['Mensagem'])
    word_probability_irrelevant[word] = prob_appearance

In [433]:
positive_review_probability = len(test_pos['Mensagem'])/len(test_all['Mensagem'])
negative_review_probability = len(test_neg['Mensagem'])/len(test_all['Mensagem'])
irrelevant_review_probability = len(test_irr['Mensagem'])/len(test_all['Mensagem'])

O classificador continua o mesmo, apenas renomeado para praticidade.

In [434]:
def Classificador_Teste(phrase) -> int:
    phrase = cleanup(phrase)
    phrase = limpaStopwords(phrase)
    words_in_review = phrase.split()
    prob_positive = 1
    prob_negative = 1
    prob_irrelevant = 1
    for word in words_in_review:
        if word in all_words_positive:
            prob_positive *= word_probability_positive[word]/word_probability_total[word]
        if word in all_words_negative:
            prob_negative *= word_probability_negative[word]/word_probability_total[word]
        if word in all_words_irrelevant:
            prob_irrelevant *= word_probability_irrelevant[word]/word_probability_total[word]

    prob_positive *= positive_review_probability
    prob_negative *= negative_review_probability
    prob_irrelevant *= irrelevant_review_probability

    chances = [prob_positive,prob_negative,prob_irrelevant]
    if prob_positive == max(chances):
        return 1
    if prob_negative == max(chances):
        return 2
    if prob_irrelevant == max(chances):
        return 0

In [435]:
bot_test = []
for review in test_all['Mensagem']:
    classification = Classificador_Treino(review)
    bot_test.append(classification)
pd.crosstab(test['Target'],bot_test,normalize=True)*100

col_0,0,1,2
Target,,,
0,18.0,0.8,0.0
1,0.0,3.6,0.0
2,4.0,20.4,53.2


Com isso, verificamos que o nosso classificador teve uma precisão de 74,6% no Dataframe de testes.

___
### Concluindo

Em conclusão, o classificador é bastante efetivo, com uma precisão bastante alta. Porém, há uma breve ideia sobre o que acontece para que o classificador não chegue em mais de 90%.

Grande parte das avaliações possui um tom de ironia em diversos aspectos. Portanto, mesmo que as palavras sejam um bom indicativo, é bastante difícil (para não dizer impossível) que seja levada em consideração também a construção da mensagem e a intenção original do autor dela.

Outro aspecto é a falta do método de Suavização de LaPlace. Por conta da forma que o código foi implementado, a suavização se tornou um processo que dificultaria não apenas o desenvolvimento, mas também o entendimento geral do projeto, quando observado nosso conhecimento atual. O método de suavização seria especialmente útil para distribuir melhor as probabilidades e evitar casos zero (onde uma probabilidade é zero), visto que sua fórmula é:

$$P'(A|B) = (Número\;de\;casos + 1)/(Número\;de\;exemplos\;totais\;do\;caso + Exemplos\;de\;casos\;virtuais)$$

Mas, no geral, o classificador foi bastante preciso em sua classificação.

O método Naive Bayes e o classificador montado podem ser utilizados, além das avaliações, para compreender alguns crescimentos gráficos e, aplicando ao mundo real, em possibilidades meteorológicas em certas regiões, como visto em uma apresentação da Universidade Federal do Paraná sobre estatística e probabilidade.

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

[Aprendizagem Bayesiana](https://www.inf.ufpr.br/menotti/ci171-182/slides/ci171-bayes.pdf) **Para compreender suavização e alguns cálculos**